In [16]:
import itertools
import random
import copy

class Minesweeper():
    """
    Minesweeper game representation
    """

    def __init__(self, height=8, width=8, mines=8):

        # Set initial width, height, and number of mines
        self.height = height
        self.width = width
        self.mines = set()

        # Initialize an empty field with no mines
        self.board = []
        for i in range(self.height):
            row = []
            for j in range(self.width):
                row.append(False)
            self.board.append(row)

        # Add mines randomly
        while len(self.mines) != mines:
            i = random.randrange(height)
            j = random.randrange(width)
            if not self.board[i][j]:
                self.mines.add((i, j))
                self.board[i][j] = True

        # At first, player has found no mines
        self.mines_found = set()

    def print(self):
        """
        Prints a text-based representation
        of where mines are located.
        """
        for i in range(self.height):
            print("--" * self.width + "-")
            for j in range(self.width):
                if self.board[i][j]:
                    print("|X", end="")
                else:
                    print("| ", end="")
            print("|")
        print("--" * self.width + "-")

    def is_mine(self, cell):
        i, j = cell
        return self.board[i][j]

    def nearby_mines(self, cell):
        """
        Returns the number of mines that are
        within one row and column of a given cell,
        not including the cell itself.
        """

        # Keep count of nearby mines
        count = 0

        # Loop over all cells within one row and column
        for i in range(cell[0] - 1, cell[0] + 2):
            for j in range(cell[1] - 1, cell[1] + 2):

                # Ignore the cell itself
                if (i, j) == cell:
                    continue

                # Update count if cell in bounds and is mine
                if 0 <= i < self.height and 0 <= j < self.width:
                    if self.board[i][j]:
                        count += 1

        return count

    def won(self):
        """
        Checks if all mines have been flagged.
        """
        return self.mines_found == self.mines




In [17]:
game1 = Minesweeper(2,2,3)
print(game1.is_mine((0,0)))
print(game1.is_mine((0,1)))
print(game1.is_mine((1,0)))
print(game1.is_mine((1,1)))
print("")
print(game1.nearby_mines((0,0)))
print(game1.nearby_mines((0,1)))
print(game1.nearby_mines((1,0)))
print(game1.nearby_mines((1,1)))

False
True
True
True

3
2
2
2


In [18]:
class Sentence():
    """
    Logical statement about a Minesweeper game
    A sentence consists of a set of board cells,
    and a count of the number of those cells which are mines.
    """

    def __init__(self, cells, count):
        self.cells = set(cells)
        self.count = count

    def __eq__(self, other):
        return self.cells == other.cells and self.count == other.count

    def __str__(self):
        return f"{self.cells} = {self.count}"

    def known_mines(self):
        """
        Returns the set of all cells in self.cells known to be mines.
        """
        # if the set contains all mines and is not an empty set
        if len(self.cells) == self.count and len(self.cells) != 0:
            return self.cells
        
        return set()

    def known_safes(self):
        """
        Returns the set of all cells in self.cells known to be safe.
        """
        # if the set is not empty and the count of its mines is zero
        if len(self.cells) != 0 and self.count == 0:
            return self.cells
        
        return set()

    def mark_mine(self, cell):
        """
        Updates internal knowledge representation given the fact that
        a cell is known to be a mine.
        """
        # if:
        first = cell in self.cells # the cell is in the sentence
        second = len(self.cells) >= 1 # the sentence isn't empty
        third = self.count >= 1 # the sentence has a mine in it
        
        if first and second and third:
            # update the sentence so that cell is no longer in the sentence
            self.cells.remove(cell)
            # and decrement the count appropriately (since removing a mine decreases the count)
            self.count -= 1
            

    def mark_safe(self, cell):
        """
        Updates internal knowledge representation given the fact that
        a cell is known to be safe.
        """
        # if:
        first = cell in self.cells # the cell is in the sentence
        second = len(self.cells) >= 1 # the sentence isn't empty
        third = len(self.cells) != self.count # the sentence doesn't consist of only mines
        
        if first and second and third:
            # update the sentence so that cell is no longer in the sentence
            # there is no need to decrement 
            self.cells.remove(cell)
        

In [19]:
sent1 = Sentence({(0,0),(0,1),(1,0),(1,1)},3)
sent2 = Sentence({(0,0),(0,1),(1,0)},2)
sent3 = Sentence({(0,1),(1,0)},2)
sent4 = Sentence({(0,0),(0,1),(1,0),(1,1)},0)
sent5 = Sentence({(0,0),(0,1),(1,0),(1,1)},4)

# test if sentences are equivalent
print(sent1)
print(sent2)
print(sent3)
print(sent4)
print(sent5)

print(sent1 == sent2)
print(sent1 == sent3)
# test if known_mines() works
print(sent1.known_mines()) # expect empty set
print(sent2.known_mines()) # expect empty set
print(sent3.known_mines()) # expect two
print(sent5.known_mines()) # expect four
print(sent1.known_safes()) # expect empty set
print(sent3.known_safes()) # expect empty set
print(sent4.known_safes()) # expect four

print(sent1.mark_mine((0,1)))
print(sent1) # expect (0,1) to be gone
print(sent3.mark_mine((0,1)))
print(sent3) # expect no change
print(sent4.mark_mine((0,1)))
print(sent4) # expect no change

print(sent1.mark_safe((0,0)))
print(sent1) # expect (0,0) to be gone
print(sent3.mark_safe((0,0)))
print(sent3) # expect no change
print(sent4.mark_safe((0,0)))
print(sent4) # expect (0,0) to be gone, no change in count


{(0, 1), (1, 0), (1, 1), (0, 0)} = 3
{(0, 1), (1, 0), (0, 0)} = 2
{(0, 1), (1, 0)} = 2
{(0, 1), (1, 0), (1, 1), (0, 0)} = 0
{(0, 1), (1, 0), (1, 1), (0, 0)} = 4
False
False
set()
set()
{(0, 1), (1, 0)}
{(0, 1), (1, 0), (1, 1), (0, 0)}
set()
set()
{(0, 1), (1, 0), (1, 1), (0, 0)}
None
{(1, 0), (1, 1), (0, 0)} = 2
None
{(1, 0)} = 1
None
{(0, 1), (1, 0), (1, 1), (0, 0)} = 0
None
{(1, 0), (1, 1)} = 2
None
{(1, 0)} = 1
None
{(0, 1), (1, 0), (1, 1)} = 0


In [20]:
print(sent1)
print(sent1.cells)
print(sent1.count)
print(len(sent1.cells))

{(1, 0), (1, 1)} = 2
{(1, 0), (1, 1)}
2
2


In [21]:
set1 = {(1,1),(2,2),(3,3)}
set2 = {(3,3),(4,4)}

diff = set1.difference(set2)


random.choice(tuple(diff))
        

(2, 2)

In [22]:
width = 4
height = 4
all_possible_moves = set()
for i in range(1,height+1):
            for j in range(1,width+1):
                all_possible_moves.add((i,j))
print(all_possible_moves)

{(4, 4), (2, 4), (1, 2), (2, 1), (3, 4), (4, 1), (3, 1), (4, 3), (1, 1), (1, 4), (4, 2), (2, 3), (3, 3), (2, 2), (3, 2), (1, 3)}


In [45]:
cell = (1,0)
cell_neighbours = []
for i in range(cell[0] - 1, cell[0] + 2):
    for j in range(cell[1] - 1, cell[1] + 2):
        
        # Ignore the cell itself
        if (i, j) == cell:
            continue
        if 0 <= i < self.height and 0 <= j < self.width:
            cell_neighbours.append((i,j))
        
print(cell_neighbours)

[(0, 0), (0, 1), (1, 1), (2, 0), (2, 1)]


In [39]:
class MinesweeperAI():
    """
    Minesweeper game player
    """

    def __init__(self, height=8, width=8):

        # Set initial height and width
        self.height = height
        self.width = width

        # Keep track of which cells have been clicked on
        self.moves_made = set()

        # Keep track of cells known to be safe or mines
        self.mines = set()
        self.safes = set()

        # List of sentences about the game known to be true
        self.knowledge = []
        
        self.all_possible_moves = set()
        for i in range(1,height+1):
            for j in range(1,width+1):
                self.all_possible_moves.add((i,j))

    def mark_mine(self, cell):
        """
        Marks a cell as a mine, and updates all knowledge
        to mark that cell as a mine as well.
        """
        self.mines.add(cell)
        for sentence in self.knowledge:
            sentence.mark_mine(cell)

    def mark_safe(self, cell):
        """
        Marks a cell as safe, and updates all knowledge
        to mark that cell as safe as well.
        """
        self.safes.add(cell)
        for sentence in self.knowledge:
            sentence.mark_safe(cell)

    def add_knowledge(self, cell, count):
        """
        Called when the Minesweeper board tells us, for a given
        safe cell, how many neighboring cells have mines in them.

        This function should:
            1) mark the cell as a move that has been made
            2) mark the cell as safe
            3) add a new sentence to the AI's knowledge base
               based on the value of `cell` and `count`
            4) mark any additional cells as safe or as mines
               if it can be concluded based on the AI's knowledge base
            5) add any new sentences to the AI's knowledge base
               if they can be inferred from existing knowledge
        """
#       1) mark the cell as a move that has been made
        self.moves_made.add(cell)
#       
#       2) mark the cell as safe
        self.mark_safe(cell)
    
#       3) add a new sentence to the AI's knowledge base based on the value of `cell` and `count`
        
        # find all neighbouring cells
        cell_neighbours = set()
        
        # loop through all cell possibilities around the target cell
        for i in range(cell[0] - 1, cell[0] + 2):
            for j in range(cell[1] - 1, cell[1] + 2):
                # ignore cells already marked as safe, mine or itself
                if (i,j) in self.safes or (i,j) in self.mines or (i, j) == cell:
                    continue
                # take the neighbours within the bounds
                if 0 <= i < self.height and 0 <= j < self.width:
                    cell_neighbours.add((i,j))
                    
        # add a new sentence to knowledge base containing the cells and how many mines there are
        self.knowledge.append(Sentence(cell_neighbours,count))                 

#       4) mark any additional cells as safe or as mines if it can be concluded based on the AI's knowledge base
        
        # go through each sentence in knowledge base
        for sentence in self.knowledge:
            mines = sentence.known_mines()
            safes = sentence.known_safes()
            
            self.safes
            # if it's safe
            if len(sentence.cells) == 1 and sentence.count == 0:
                sentence.mark_safe(sentence.cells)
            # if it's a mine
            if len(sentence.cells) == 1 and sentence.count >= 1:
                sentence.mark_mine(sentence.cells)
                
#       5) add any new sentences to the AI's knowledge base
#          if they can be inferred from existing knowledge

#       5a) By extension, any time we have a sentence whose count is 0, we know that 
#           all of that sentence’s cells must be safe.
        
        # go through each sentence in knowledge base
        for sentence in self.knowledge:
            # if the sentence has a count of zero
            if sentence.count == 0:
                # all the sentence's cells are safe
                for cell in sentence.cells:
                    sentence.mark_safe(cell)

#       5b) More generally, any time the number of cells is equal to the count, we know that 
#           all of that sentence’s cells must be mines
        
        # go through each sentence in knowledge base
        for sentence in self.knowledge:
            # if the number of cells is equal to the count
            if len(sentence.cells) == sentence.count:
                # all of the sentence's cells are mines
                for cell in sentence.cells:
                    sentence.mark_mine(cell)
        
#       5c) More generally, any time we have two sentences set1 = count1 and set2 = count2 
#           where set1 is a subset of set2, then we can construct the new sentence set2 - set1 = count2 - count1
        
        # go through each sentence in knowledge base
        for sentence1 in self.knowledge:
            # compare it against each other sentence in knowledge base
            for sentence2 in self.knowledge:
                if sentence1 != sentence2:
                    # if set1 is a subset of set2
                    if sentence1.cells.issubset(sentence2.cells):
                        # construct the new sentence set2 - set1 = count2 - count1
                        self.knowledge.add(Sentence(
                            cells = sentence2.cells.difference(sentence1.cells), 
                            count = sentence2.count - sentence1.count))

    def make_safe_move(self):
        """
        Returns a safe cell to choose on the Minesweeper board.
        The move must be known to be safe, and not already a move
        that has been made.

        This function may use the knowledge in self.mines, self.safes
        and self.moves_made, but should not modify any of those values.
        """
        # if there are no safe moves, return nothing
        if len(self.safes) == 0:
            return None
        # otherwise, find the set of safe moves you can make that you haven't already
        diff = self.safes.difference(self.moves_made)

        # choose an item from the set and return it (convert to tuple to be able to use random.choice)
        return random.choice(tuple(diff))


    def make_random_move(self):
        """
        Returns a move to make on the Minesweeper board.
        Should choose randomly among cells that:
            1) have not already been chosen, and
            2) are not known to be mines
        """
        # if there are no possible moves to make
        if len(all_moves) == len(self.moves_made):
            return None
        
        # all moves possible were calculated on init based on size of board (height, width)
        all_moves = self.all_possible_moves

        # illegal moves are those that have already been made or are mines
        illegal_moves = self.moves_made.union(self.mines)

        # allowed moves are those on the board that aren't "illegal"
        allowed_moves = all_moves.difference(illegal_moves)

        # randomly pick one move to make from the set
        return random.choice(tuple(allowed_moves))


In [42]:
game2 = MinesweeperAI(3,3)
print(game2.height)
print(game2.width)
print(game2.moves_made)
print(game2.mines)
print(game2.safes)
print(game2.knowledge)
print(game2.all_possible_moves)

print("")

game2.mark_safe((1,1))
print(game2.moves_made)
print(game2.mines)
print(game2.safes)
print(game2.knowledge)

print("")

game2.mark_mine((1,2))
print(game2.moves_made)
print(game2.mines)
print(game2.safes)
print(game2.knowledge)

print("")

game2.make_safe_move()
print(game2.moves_made)
print(game2.mines)
print(game2.safes)
print(game2.knowledge)

3
3
set()
set()
set()
[]
{(1, 2), (2, 1), (3, 1), (1, 1), (2, 3), (3, 3), (2, 2), (3, 2), (1, 3)}

set()
set()
{(1, 1)}
[]

set()
{(1, 2)}
{(1, 1)}
[]

{(1, 1)}
{(1, 1)}
set()
{(1, 2)}
{(1, 1)}
[]
